# 저장했던 db 불러오기

In [ ]:
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import MetaData
from sqlalchemy import Table, Column, Integer, String, ForeignKey
from sqlalchemy import select, update, join
from sqlalchemy import and_, or_, not_

In [ ]:
engine = create_engine("sqlite:///alchemytest.db", echo=True)
# 엔진에서 connection 만듦.
conn = engine.connect()

In [ ]:
# 기존에 engine에 있는 db정보를 reflect해서 가져와서 metadata에 바인딩할거임.

metadata = MetaData(bind=engine, reflect=True)

In [ ]:
# reflect=True 옵션쓰지말고, metadata에서reflect() 함수를 부르랭.
# metadata에 engine에있는 애들을 싹 가져왔음.
metadata.reflect(bind=engine)

In [ ]:
print(metadata.tables)

In [ ]:
for row in metadata.tables:
    print(row)

In [ ]:
album = metadata.tables["Album"]

In [ ]:
result = conn.execute(album.select())
print(result)

In [ ]:
for row in result:
    print(row)

# SQLAlchemy ORM

In [1]:
# base : metadata와 비슷한 역할하는 애.
# metadata를 상속받아서 tabel을 만들었다면, bse를 상속받아 class를 만ㄷ르거임.
# 엔진은 항상 필요함.

from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String, ForeignKey
engine = create_engine("sqlite:///", echo=True)
from sqlalchemy.ext.declarative import declarative_base

In [2]:
Base = declarative_base()

In [3]:
Base.metadata.tables

immutabledict({})

In [ ]:
# Base.metadata.drop_all(engine)

In [ ]:
# Base.metadata.drop_all(engine)
# Base.__table__.drop(Artist)
# Base.__table__.drop(Album)
# Base.__table__.drop(Genre)
# Base.__table__.drop(Track)

In [4]:
# 앞으로 engine은 얘를 참조할것임.

class Artist(Base):
    __tablename__ = "artist"
    id = Column(Integer, primary_key=True)
    # class 선언.
    # 이걸로 원래 끝인데, 상석 어쩌고 상세설정도 가능함...
    name = Column(String)
    extend_existing=True
    #artist를 찍으면 print문이 실행될것임.
    def __repr__(self):
        return "<T'artist(name='%s')>" %(self.name)

In [5]:
class Album(Base):
    __tablename__ = "album"
    id = Column(Integer, primary_key=True)
    title = Column(String)
    extend_existing=True
    artist_id = Column(Integer, ForeignKey("artist.id"))
    

In [6]:
class Genre(Base):
    __tablename__ = "genre"
    id = Column(Integer, primary_key=True)
    name = Column(String)
    extend_existing=True
    

In [7]:
class Genre(Base):
    __tablename__ = "genre"
    id = Column(Integer, primary_key=True)
    name = Column(String)
    extend_existing=True
    class Track(Base):
    __tablename__ = "track"
    id = Column(Integer, primary_key=True)
    title = Column(String)
    extend_existing=True
    album_id = Column(Integer, ForeignKey("album.id"))
    genre_id = Column(Integer, ForeignKey("genre.id"))

In [9]:
#아까만든 엔진 바인딩해주기.
# 클래스만든것밖에 없는데 지가알아서 sql문 만듦.

Base.metadata.create_all(engine)

2018-07-10 17:58:35,850 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-07-10 17:58:35,853 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 17:58:35,855 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-07-10 17:58:35,856 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 17:58:35,858 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("artist")
2018-07-10 17:58:35,859 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 17:58:35,861 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("album")
2018-07-10 17:58:35,862 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 17:58:35,863 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("genre")
2018-07-10 17:58:35,864 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 17:58:35,865 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("track")
2018-07-10 17:58:35,866 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 17:58:35,868 INFO sqlalchemy.engin

In [10]:
artist1 = Artist(name="Led Zepplin")
artist2 = Artist(name="AC/DC")

In [11]:
artist1

<T'artist(name='Led Zepplin')>

In [12]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)

In [13]:
session = Session()

# metadata 없는 대신, base가 있고,
# 연결하기위해 conn없는 대신 session이 있음

In [14]:
# session에 일을 시킬것임.
session.add_all([artist1, artist2])

In [15]:
session.commit()

# session을통해 bind딩된 engine을 통해서, add시켜달라고 명령하고. 실제 db에 물리적으로 저장하는 단계까지!

2018-07-10 17:58:52,670 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-07-10 17:58:52,673 INFO sqlalchemy.engine.base.Engine INSERT INTO artist (name) VALUES (?)
2018-07-10 17:58:52,674 INFO sqlalchemy.engine.base.Engine ('Led Zepplin',)
2018-07-10 17:58:52,675 INFO sqlalchemy.engine.base.Engine INSERT INTO artist (name) VALUES (?)
2018-07-10 17:58:52,676 INFO sqlalchemy.engine.base.Engine ('AC/DC',)
2018-07-10 17:58:52,677 INFO sqlalchemy.engine.base.Engine COMMIT


In [16]:
session.dirty

IdentitySet([])

In [17]:
session.new

IdentitySet([])

In [18]:
artist1.name = 'xyz'
session.dirty

# 변경되었는데 반영되지 않은 값이 있다는 것을 알려줌. commit해야함. 미반영된 애들을 sql로 변형해서 반영하게됨.

IdentitySet([<T'artist(name='xyz')>])

In [19]:
# 리스트 안에 class를 쓴다는점이 아까랑 다름. album에다가 하나의 인스턴스를 추가하려함.

session.add_all([
    Album(title="1", artist_id=1),
    Album(title="2", artist_id=2)
])

# 값이 바뀐게 없음. attribute가 수정된게 아니라, 메모리상에만 있는값임.
# commit 해야 반영됨.

In [20]:
session.add_all([
    Genre(name="Rock"),
    Genre(name="Metal")
])

In [21]:
#session에 아무작업안하고 붙여넣기만함.

session.add_all([
    Track(title="1-1", album_id=1, genre_id=1),
    Track(title="1-2", album_id=1, genre_id=2),
    Track(title="2-1", album_id=2, genre_id=1),
    Track(title="2-2", album_id=2, genre_id=2)
])

# session.commit 이랑, session.new하면 여태까지 무슨일했는지 쫙 찍어줌.
# SQL문이랑 전혀 상관없잖아!!! 굉장히 괜찮지않닝?ㅋㅋㅋㅋㅋㅋㅋㅋㅋ
# commit이랑 new, dirty로 확인하면서 그냥 너무나쉽게 OOP로 database관리하면 됨.

In [22]:
temp = Track(title="3-1", album_id=3, genre_id=3)

In [23]:
print(temp.id)

# None이라고 나옴. 실제DB에 아직 안쓰고 메모리상에만 존재하기때문. session에다가도 아직 안넣은상태.

None


In [24]:
session.add(temp)

In [25]:
session.new

# DB에 아직 기록이안되어있으니까 id를 못불러오는 상황임.

IdentitySet([<__main__.Album object at 0x00000134DDD66B70>, <__main__.Album object at 0x00000134DDD66BE0>, <__main__.Genre object at 0x00000134DDD66C50>, <__main__.Genre object at 0x00000134DDD66CC0>, <__main__.Track object at 0x00000134DDD86400>, <__main__.Track object at 0x00000134DDD86470>, <__main__.Track object at 0x00000134DDD864E0>, <__main__.Track object at 0x00000134DDD86550>, <__main__.Track object at 0x00000134DDD66FD0>])

In [26]:
session.commit()

2018-07-10 17:59:11,889 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-07-10 17:59:11,891 INFO sqlalchemy.engine.base.Engine SELECT artist.id AS artist_id 
FROM artist 
WHERE artist.id = ?
2018-07-10 17:59:11,892 INFO sqlalchemy.engine.base.Engine (1,)
2018-07-10 17:59:11,894 INFO sqlalchemy.engine.base.Engine UPDATE artist SET name=? WHERE artist.id = ?
2018-07-10 17:59:11,894 INFO sqlalchemy.engine.base.Engine ('xyz', 1)
2018-07-10 17:59:11,896 INFO sqlalchemy.engine.base.Engine INSERT INTO genre (name) VALUES (?)
2018-07-10 17:59:11,897 INFO sqlalchemy.engine.base.Engine ('Rock',)
2018-07-10 17:59:11,898 INFO sqlalchemy.engine.base.Engine INSERT INTO genre (name) VALUES (?)
2018-07-10 17:59:11,899 INFO sqlalchemy.engine.base.Engine ('Metal',)
2018-07-10 17:59:11,901 INFO sqlalchemy.engine.base.Engine INSERT INTO track (title, album_id, genre_id) VALUES (?, ?, ?)
2018-07-10 17:59:11,902 INFO sqlalchemy.engine.base.Engine ('1-1', 1, 1)
2018-07-10 17:59:11,904 INFO sqlalchemy

In [27]:
print(temp.id)

# ORM이니까 가능한것!! 엄청나게 간단하게 제공해주고있음....!! 정말 획기적인것임ㅋ

2018-07-10 17:59:15,004 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-07-10 17:59:15,006 INFO sqlalchemy.engine.base.Engine SELECT track.id AS track_id, track.title AS track_title, track.album_id AS track_album_id, track.genre_id AS track_genre_id 
FROM track 
WHERE track.id = ?
2018-07-10 17:59:15,007 INFO sqlalchemy.engine.base.Engine (5,)
5


In [28]:
# 데이터를 불러올려고 할 때는 어떻게 하느냐?
# 실제 데이터에 접근할 일은 그렇게 많지 않음. 이미session에 올라와있고 DB와 완벽하게 100% 일치함.
# data가져오기위해 DB에 다녀오는것보다, 메모리상에 있는 데이터 갖다주는게 훨씬 빠름..!!!!
# 따라서 session에 있는 데이터 가져오게 함으로서 기가막힌거랭!!

for row in session.query(Artist):
    print(row.id, row.name)

2018-07-10 17:59:16,784 INFO sqlalchemy.engine.base.Engine SELECT artist.id AS artist_id, artist.name AS artist_name 
FROM artist
2018-07-10 17:59:16,785 INFO sqlalchemy.engine.base.Engine ()
1 xyz
2 AC/DC


In [29]:
for row in session.query(Artist).filter(Artist.id == 1):
    print(row.id, row.name)

2018-07-10 17:59:18,426 INFO sqlalchemy.engine.base.Engine SELECT artist.id AS artist_id, artist.name AS artist_name 
FROM artist 
WHERE artist.id = ?
2018-07-10 17:59:18,427 INFO sqlalchemy.engine.base.Engine (1,)
1 xyz


In [30]:
# session을 통해 engine을 바인딩시키기.
# 실제 instance를 반영시키기.
# nes, dirty를 쓰면 수정사항확인되는거고.
# 실제 commit했을 때 반영되는거고.
# query로 한번 감싸는거
# filter쓰는법.(두종류가 있음.)
# (class안에있는 attribute위주로 설명하는거고/ string임. 실제 테이블 안에 keyword로 있냐없냐 판단할때 사용.)